In [2]:
import pandas as pd
base = pd.read_csv('house_prices.csv')
base.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [4]:
base.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [3]:
colunas_usadas = ['price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']
colunas_usadas

['price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long']

In [7]:
#Recarregando a base de dados utilizando somente as colunas indicadas

base = pd.read_csv('house_prices.csv', usecols = colunas_usadas)

In [10]:
# Aplicação da normalização nos atributos previsores

from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()

base[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']] = scaler_x.fit_transform(base[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']])

scaler_y = MinMaxScaler()
base[['price']] = scaler_y.fit_transform(base[['price']]) ## normalizando o Y

In [12]:
x = base.drop('price', axis=1) # atributos previsores
y = base.price # variável dependente

In [13]:
previsores_colunas = colunas_usadas[1:17]
previsores_colunas

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long']

In [14]:
import tensorflow as tf

C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarnin

In [17]:
# Feature colunm (recebendo o nome das colunas)
colunas = [tf.feature_column.numeric_column(key = c) for c in previsores_colunas] 

In [18]:
from sklearn.model_selection import train_test_split # separando base de dados, 70% treinamento e 30% teste
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x,y, test_size = 0.3)

In [19]:
funcao_treinamento = tf.estimator.inputs.pandas_input_fn(x = x_treinamento, y = y_treinamento,
                                                        batch_size = 32, num_epochs = None, shuffle = True)

In [22]:
regressor = tf.estimator.DNNRegressor(hidden_units = [8, 8, 8], feature_columns = colunas)
regressor.train(input_fn = funcao_treinamento, steps = 20000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Douglas\\AppData\\Local\\Temp\\tmp1t2qxetm', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001A7FE282A48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Var

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Douglas\AppData\Local\Temp\tmp1t2qxetm\model.ckpt.
INFO:tensorflow:loss = 0.16324124, step = 1
INFO:tensorflow:global_step/sec: 342.438
INFO:tensorflow:loss = 0.012115644, step = 101 (0.297 sec)
INFO:tensorflow:global_step/sec: 314.444
INFO:tensorflow:loss = 0.012751382, step = 201 (0.316 sec)
INFO:tensorflow:global_step/sec: 328.923
INFO:tensorflow:loss = 0.05672551, step = 301 (0.304 sec)
INF

INFO:tensorflow:global_step/sec: 384.585
INFO:tensorflow:loss = 0.017743312, step = 6101 (0.259 sec)
INFO:tensorflow:global_step/sec: 387.568
INFO:tensorflow:loss = 0.009757232, step = 6201 (0.260 sec)
INFO:tensorflow:global_step/sec: 389.078
INFO:tensorflow:loss = 0.026181635, step = 6301 (0.258 sec)
INFO:tensorflow:global_step/sec: 390.596
INFO:tensorflow:loss = 0.008291952, step = 6401 (0.254 sec)
INFO:tensorflow:global_step/sec: 384.586
INFO:tensorflow:loss = 0.012002831, step = 6501 (0.261 sec)
INFO:tensorflow:global_step/sec: 399.969
INFO:tensorflow:loss = 0.0118129505, step = 6601 (0.250 sec)
INFO:tensorflow:global_step/sec: 392.129
INFO:tensorflow:loss = 0.010984419, step = 6701 (0.254 sec)
INFO:tensorflow:global_step/sec: 401.576
INFO:tensorflow:loss = 0.0064321356, step = 6801 (0.249 sec)
INFO:tensorflow:global_step/sec: 386.072
INFO:tensorflow:loss = 0.010198049, step = 6901 (0.259 sec)
INFO:tensorflow:global_step/sec: 384.587
INFO:tensorflow:loss = 0.021779902, step = 7001 

INFO:tensorflow:global_step/sec: 392.127
INFO:tensorflow:loss = 0.0067240763, step = 14201 (0.255 sec)
INFO:tensorflow:global_step/sec: 370.343
INFO:tensorflow:loss = 0.007950624, step = 14301 (0.271 sec)
INFO:tensorflow:global_step/sec: 359.685
INFO:tensorflow:loss = 0.0047381753, step = 14401 (0.277 sec)
INFO:tensorflow:global_step/sec: 362.293
INFO:tensorflow:loss = 0.007740146, step = 14501 (0.277 sec)
INFO:tensorflow:global_step/sec: 390.565
INFO:tensorflow:loss = 0.024633875, step = 14601 (0.254 sec)
INFO:tensorflow:global_step/sec: 389.107
INFO:tensorflow:loss = 0.009041701, step = 14701 (0.257 sec)
INFO:tensorflow:global_step/sec: 390.597
INFO:tensorflow:loss = 0.010912928, step = 14801 (0.256 sec)
INFO:tensorflow:global_step/sec: 398.376
INFO:tensorflow:loss = 0.032305457, step = 14901 (0.251 sec)
INFO:tensorflow:global_step/sec: 381.651
INFO:tensorflow:loss = 0.005048139, step = 15001 (0.263 sec)
INFO:tensorflow:global_step/sec: 389.021
INFO:tensorflow:loss = 0.01785829, step

In [23]:
funcao_previsao = tf.estimator.inputs.pandas_input_fn(x = x_teste, shuffle = False)
previsoes = regressor.predict(input_fn = funcao_previsao)
list(previsoes)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\Douglas\AppData\Local\Temp\tmp1t2qxetm\model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'predictions': array([0.02144917], dtype=float32)},
 {'predictions': array([0.06073497], dtype=float32)},
 {'predictions': array([0.06434701], dtype=float32)},
 {'predictions': array([0.06354669], dtype=float32)},
 {'predictions': array([0.05018071], dtype=float32)},
 {'predictions': array([0.03608415], dtype=float32)},
 {'predictions': array([0.07836093], dtype=float32)},
 {'predictions': array([0.0724363], dtype=float32)},
 {'predictions': array([0.09341861], dtype=float32)},
 {'predictions': array([0.06603931], dtype=float32)},
 {'predictions': array([0.10811175], dtype=float32)},
 {'predictions': array([0.04206864], dtype=float32)},
 {'predictions': array([0.01645407], dtype=float32)},
 {'predictions': array([0.09292081], dtype=float32)},
 {'predictions': array([0.07345928], dtype=float32)},
 {'predictions': array([0.05300603], dtype=float32)},
 {'predictions': array([0.07362856], dtype=float32)},
 {'predictions': array([0.043029], dtype=float32)},
 {'predictions': array([0.03916

In [24]:
valores_previsoes = []
for p in regressor.predict(input_fn = funcao_previsao):
    valores_previsoes.append(p['predictions'][0])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Douglas\AppData\Local\Temp\tmp1t2qxetm\model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [25]:
valores_previsoes

[0.021449171,
 0.06073497,
 0.064347014,
 0.06354669,
 0.050180707,
 0.036084145,
 0.07836093,
 0.072436295,
 0.09341861,
 0.06603931,
 0.108111754,
 0.04206864,
 0.016454069,
 0.09292081,
 0.07345928,
 0.05300603,
 0.07362856,
 0.043029003,
 0.039168082,
 0.034197345,
 0.06853177,
 0.087518394,
 0.090837,
 0.034727946,
 0.07313735,
 0.02326823,
 0.04932485,
 0.030845419,
 0.03331764,
 0.07426414,
 0.021338675,
 0.07132554,
 0.06472023,
 0.12351805,
 0.043737907,
 0.04069672,
 0.06519166,
 0.053472567,
 0.054525886,
 0.042445663,
 0.050426353,
 0.07654126,
 0.044554688,
 0.079991415,
 0.058543034,
 0.07094623,
 0.02995213,
 0.052595228,
 0.09350467,
 0.037292928,
 0.023550168,
 0.019627279,
 0.0670331,
 0.07346137,
 0.060500268,
 0.33606884,
 0.023585424,
 0.038358822,
 0.02029204,
 0.051974915,
 0.032479357,
 0.14959495,
 0.03901479,
 0.02080411,
 0.05750844,
 0.0672836,
 0.047604542,
 0.100234956,
 0.07341325,
 0.074898705,
 0.06635125,
 0.04129256,
 0.03976441,
 0.049452435,
 0.0335

In [26]:
import numpy as np
valores_previsoes = np.asarray(valores_previsoes).reshape(-1,1)
valores_previsoes = scaler_y.inverse_transform(valores_previsoes)
valores_previsoes

array([[238549.94],
       [538104.1 ],
       [565645.94],
       ...,
       [833431.06],
       [263298.2 ],
       [832873.5 ]], dtype=float32)

In [28]:
y_teste2 = y_teste.values.reshape(-1,1)
y_teste2 = scaler_y.inverse_transform(y_teste2)
y_teste2

array([[322000.],
       [660000.],
       [650000.],
       ...,
       [697000.],
       [285000.],
       [847000.]])

In [29]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_teste2, valores_previsoes)
mae

92980.98711491749

In [30]:
# se observa em relação a outro exemplo sem usar redes neurais que o valor do erro foi de 130k

# ganho considerável